In [1]:
import warnings

import pandas as pd
import numpy as np
import xarray as xr

from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from ptsa.data.timeseries import TimeSeries
from cmlreaders import CMLReader, get_data_index

In [2]:
import Reader
import ComputePower

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
warnings.filterwarnings('ignore')

In [5]:
EXPERIMENT = 'TH1'
EVENTS_TYPE = 'CHEST'
REL_START=-2250
REL_STOP=3750
BUFFER=2000
FREQ_BAND=[1,3]

In [6]:
def error_fill(xs, ys, err, color, label, axes=None):
    
    if axes == None:
        plotter = plt
    else:
        plotter = axes
    
    plotter.fill_between(xs, ys-err, ys+err,
                     alpha=.4, color=color)
    plotter.plot(xs, ys, label=label, color=color)

In [7]:
def get_roi_locs(subjects):

    warnings.filterwarnings('ignore')

    rois = ['Left EC', 'Right EC', 'Left CA1', 'Right CA1']

    roi_locs = {}
    for roi in rois:
        roi_locs[roi] = []

    for i, subject in subjects.iterrows():
        contacts = Reader.load_contacts(**subject)
        for roi in rois:
            if roi in contacts['stein.region'].unique():
                roi_locs[roi].append(True)
            else:
                roi_locs[roi].append(False)

    for roi in rois:
        roi_locs[roi] = np.array(roi_locs[roi])

    return roi_locs

In [8]:
def get_erps_1subj(region, freq_band,
                   events_type=EVENTS_TYPE, 
                   rel_start_ms=REL_START, rel_stop_ms=REL_STOP,
                   window_size=None,
                   buf_ms=BUFFER, resample_freq=None,
                   transform='hilbert',
                   eeg_kwargs={}, wave_kwargs={}, pow_kwargs={},
                   **subject_dict
                  ):
    # organize
    pow_kwargs['window_size'] = window_size
    pow_kwargs['transform'] = transform
    eeg_kwargs['resample_freq'] = resample_freq
    
    # get events
    events = Reader.get_all_events(**subject_dict)
    events = events[events['type']==events_type]
    events.index = range(len(events))
    
    # get elecs
    all_elecs = Reader.load_contacts(**subject_dict)
    contacts = Reader.get_region_contacts(region, 'stein.region', all_elecs)
    if len(contacts)==0:
        raise ValueError(f"subject {subject_dict} has no electrodes in region {region}")
        
    # get baseline power
    basepow = ComputePower.get_basepow(
        events=events, freq_band=freq_band,
        which_contacts=contacts, buf_ms=buf_ms,
        subject_dict=subject_dict, wave_kwargs=wave_kwargs,
        eeg_kwargs=eeg_kwargs, pow_kwargs=pow_kwargs,
    )
    
    # get events eeg
    eeg = Reader.load_eeg(
        events, contacts, **subject_dict,
        rel_start_ms=rel_start_ms, rel_stop_ms=rel_stop_ms,
        do_average_ref=False, buf_ms=buf_ms,
        resample_freq=resample_freq,
    )
    
    # get events powers
    powers = ComputePower.compute_power(eeg, freq_band=freq_band, buf_ms=buf_ms,
                                        **wave_kwargs, **pow_kwargs,
                                       )
    
    # zscore events powers using baseline powers
    powers = ComputePower.zscore_powers(powers, basepow)
    
    # split by recalled, not recalled, and empty chests
    recalled_locs = events['recalled'].values
    empty_locs = (events['item_name']=='').values
    not_recalled_locs = ~(recalled_locs|empty_locs)
    
    # get powers accross events
    rec_power = powers[recalled_locs].mean(dim='event')
    nrec_power = powers[not_recalled_locs].mean(dim='event')
    empty_power = powers[empty_locs].mean(dim='event')
    
    # get erps accross channels
    rec_erp = rec_power.mean(dim='channel')
    rec_sem = rec_power.std(dim='channel') / np.sqrt(rec_power.channel.size)

    nrec_erp = nrec_power.mean(dim='channel')
    nrec_sem = nrec_power.std(dim='channel') / np.sqrt(nrec_power.channel.size)

    empty_erp = empty_power.mean(dim='channel')
    empty_sem = empty_power.std(dim='channel') / np.sqrt(nrec_power.channel.size)

    # done
    return pd.DataFrame({'rec_erp': rec_erp, 'rec_sem': rec_sem,
                         'nrec_erp': nrec_erp, 'nrec_sem': nrec_sem,
                         'empty_erp': empty_erp, 'empty_sem': empty_sem, 
                         'samplerate': rec_power.samplerate.values[()],
                         'window_size': window_size,
                         'freq_band': str(freq_band)
                        }, index=rec_erp.time)

In [9]:
def get_erps(subjects, region, freq_band,
             graphit=True, resample_freq=500,
             window_size=None,
             **erp_kwargs):

    if graphit:
        # set up graph on 1 subplot per subj
        ncols = int(np.sqrt(len(subjects)))
        nrows = int(np.ceil(len(subjects)/ncols))
        fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,15))
        fig.tight_layout()

    # store each subjects erps in these lists
    all_recs = []
    all_nrecs = []
    all_emptys = []
    subjects_with_data = []

    # get each subject's erp
    for i, subject in tqdm(subjects.iterrows(), total=len(subjects)):

        try:
            erps = get_erps_1subj(
                **subject, region=region,
                freq_band=freq_band,
                resample_freq=resample_freq,
                window_size=window_size,
                **erp_kwargs
            )

            all_recs.append(erps['rec_erp'])
            all_nrecs.append(erps['nrec_erp'])
            all_emptys.append(erps['empty_erp'])
            subjects_with_data.append(
                Reader.subject_id(subject['subject'], subject['montage']))

            if graphit:
                plot_erp(erps, axes=axes[i%nrows, int(i/nrows)], do_legend=False)
        except (IndexError, ValueError):
            continue
            
    if graphit:
        plt.show()
    
    # concatenate erps accross subjects 
    recs = pd.concat(all_recs, axis=1)
    recs = xr.DataArray(recs.values.transpose(),
                        coords=[subjects_with_data, recs.index],
                        dims = ['subject', 'time'])

    nrecs = pd.concat(all_nrecs, axis=1)
    nrecs = xr.DataArray(nrecs.values.transpose(),
                        coords=[subjects_with_data, nrecs.index],
                        dims = ['subject', 'time'])

    emptys = pd.concat(all_emptys, axis=1)
    emptys = xr.DataArray(emptys.values.transpose(),
                        coords=[subjects_with_data, emptys.index],
                        dims = ['subject', 'time'])
    
    # get erps accross subjects
    rec_erp = recs.mean(dim='subject')
    rec_sem = recs.std(dim='subject') / np.sqrt(recs.subject.size)

    nrec_erp = nrecs.mean(dim='subject')
    nrec_sem = nrecs.std(dim='subject') / np.sqrt(nrecs.subject.size)

    empty_erp = emptys.mean(dim='subject')
    empty_sem = emptys.std(dim='subject') / np.sqrt(emptys.subject.size)
    
    # done
    return pd.DataFrame({'rec_erp': rec_erp, 'rec_sem': rec_sem,
                         'nrec_erp': nrec_erp, 'nrec_sem': nrec_sem,
                         'empty_erp': empty_erp, 'empty_sem': empty_sem, 
                         'samplerate': resample_freq, 'window_size': window_size,
                         'freq_band': f'{freq_band[0]}-{freq_band[1]}'
                        }, index=rec_erp.time)

In [10]:
def plot_erp(erps,
             rel_start_ms=REL_START, rel_stop_ms=REL_STOP,
             axes=None, do_legend=True):
    
    samplerate = erps['samplerate'].iloc[0]
    freq_band = erps['freq_band'].iloc[0]
    window_size = erps['window_size'].iloc[0]
    
    xs = erps.index
    if window_size is not None:
        xs -= window_size/2
    xs /= 1000
    
    rec_color = 'firebrick'
    nrec_color = 'steelblue'
    empty_color = 'gray'
    
    if axes is None:
        axes = plt.subplot()

    error_fill(xs, erps['rec_erp'], erps['rec_sem'], color=rec_color, label='recalled', axes=axes)
    error_fill(xs, erps['nrec_erp'], erps['nrec_sem'], color=nrec_color, label='not recalled', axes=axes)
    error_fill(xs, erps['empty_erp'], erps['empty_sem'], color=empty_color, label='empty', axes=axes)

    axes.axvline(0, color='k')
    axes.axvline(1.5, color='k', linestyle=':') # item stops being displayed
    axes.axhline(0, color='k', linestyle='--')

    axes.set_xlabel('time (s)')
    axes.set_ylabel(fr'Z(power) $\emdash$ [{freq_band}] Hz')

    if do_legend:
        axes.legend()
    
    #axes.set_ylim((-.2,.3))
    axes.set_xlim((-2.5,4))

In [11]:
region='Right CA1'

In [12]:
df = get_data_index('r1')
df = df[df['experiment']==EXPERIMENT]
subjects = df[['subject', 'montage', 'localization', 'experiment']].drop_duplicates()
subjects.index = range(len(subjects))

In [13]:
subject = subjects.iloc[12]

erps = get_erps_1subj(
    **subject, region=region,
    freq_band=[1,3],
    resample_freq=None,
    window_size=500,
    transform='wavelet'
)

TypeError: load_eeg() missing 1 required positional argument: 'contacts'

In [ ]:
regions = ['Left CA1', 'Right CA1']
freq_bands = [[40, 100], [3,10], [1,3]]

fig, axes = plt.subplots(nrows=len(freq_bands), ncols=len(regions), figsize=(10,20))

for col, region in enumerate(regions):
    for row, freq_band in enumerate(freq_bands):
        ax = axes[row, col]
        
        erps = get_erps(subjects, region=region,
                        freq_band=freq_band, graphit=False,
                        transform='wavelet', window_size=500,
                       )
        plot_erp(erps, axes=ax)
        
        if row == 0:
            ax.set_title(region)

In [ ]:
erps = get_erps(subjects, region='Left CA1',
                        freq_band=[1,3], graphit=False,
                        transform='wavelet', window_size=500,
                       )
plot_erp(erps, axes=ax)

In [ ]:
plot_erp(erps)